In [77]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification


In [95]:

raw_shelves_df = pd.read_csv('shelves.csv')
raw_shedules_df = pd.read_csv('shedules.csv')
raw_products_df = pd.read_csv('products.csv')


,id_tov,Наименование,Категория,Группа,Кол-во чеков за месяц,Единица измерения,Кол-во за месяц,Среднее кол-во за месяц,Сумма продаж за месяц
0,23 074,Манго Египет,Овощи. Фрукты. Грибы. Зелень,Манго (все виды),8352,кг,6972.48,0.83,2194135.41
1,731,Бананы,Овощи. Фрукты. Грибы. Зелень,Банан,6869,кг,6338.46,0.92,952972.7
2,647,"Пакет-майка ""ВкусВилл"" малый",Нон-Фуд,"Сумки, пакеты, упаковка",87843,шт,96923.00,1.10,756861.36
3,19 419,Бедро цыпленка-бройлера бескостное (филе бедра),Мясо. Птица. П/ф охл. Яйца,Курица бескостное мясо,1504,кг,1438.22,0.96,723763.71
4,669,Арбуз,Овощи. Фрукты. Грибы. Зелень,Бахчевые,1621,кг,11265.38,6.95,683772.7


In [91]:
shedules_df = raw_shedules_df[raw_shedules_df['name_TT'].str.contains("5018ДС_СПб_Полюстровский59", case=False, na=False)]

shedules_df = shedules_df.rename(columns={
    'N': 'id',
    'name_TT': 'store',
    'НаименованиеГрафика': 'schedule_name',
    'Длительность (недель)': 'duration_weeks',
    'Стеллажи': 'shelves',
    'неделя 1': 'week_1',
    'неделя 2': 'week_2',
    'неделя 3': 'week_3',
    'неделя 4': 'week_4',
    'Кол-во дней': 'days_count',
    'ЧастотаПостеллажная': 'shelf_frequency',
    'ЧастотаГрафика': 'schedule_frequency'
}, inplace=False)

shedules_df = shedules_df.reset_index(drop=True, inplace=False)

shedules_df.head()

,id,store,schedule_name,duration_weeks,shelves,week_1,week_2,week_3,week_4,days_count,shelf_frequency,schedule_frequency
0,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,2,A1;A10;A10;A10;A10;A10;A10;A10;A10;A11;A11;A11...,пн,пн,NaN,NaN,2,0.14,0.93
1,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,2,G13;O1;O2;O3;O4;O5;O6;O7;O8;Г1;Г10;Г11;Г12;Г2;...,ср,ср,NaN,NaN,2,0.14,0.93
2,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,2,K1;K10;K11;K12;K13;K14;K15;K16;K17;K18;K19;K2;...,вт,вт,NaN,NaN,2,0.14,0.93
3,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,2,Z1;Z10;Z12;Z13;Z14;Z15;Z16;Z17;Z18;Z2;Z3;Z4;Z5...,чт,чт,NaN,NaN,2,0.14,0.93
4,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,2,Z18;Z19;Z20;Z21;Z22;Z23;Z24;Z25;Z26;Z27;Z28;Z2...,пт,NaN,NaN,NaN,1,0.07,0.93


In [97]:
products_df = raw_products_df

products_df = products_df.rename(columns={
    'id_tov': 'id_tov',
    'Наименование': 'name',
    'Категория': 'category',
    'Группа': 'group',
    'Кол-во чеков за месяц': 'monthly_hours',
    'Единица измерения': 'system',
    'Кол-во за месяц': 'per_month',
    'Среднее кол-во за месяц': 'avg_per_month',
    'Сумма продаж за месяц': 'sum_per_month',
}, inplace=False)

# shedules_df = shedules_df.reset_index(drop=True, inplace=False)

products_df.head()

,id_tov,name,category,group,monthly_hours,system,per_month,avg_per_month,sum_per_month
0,23 074,Манго Египет,Овощи. Фрукты. Грибы. Зелень,Манго (все виды),8352,кг,6972.48,0.83,2194135.41
1,731,Бананы,Овощи. Фрукты. Грибы. Зелень,Банан,6869,кг,6338.46,0.92,952972.7
2,647,"Пакет-майка ""ВкусВилл"" малый",Нон-Фуд,"Сумки, пакеты, упаковка",87843,шт,96923.00,1.10,756861.36
3,19 419,Бедро цыпленка-бройлера бескостное (филе бедра),Мясо. Птица. П/ф охл. Яйца,Курица бескостное мясо,1504,кг,1438.22,0.96,723763.71
4,669,Арбуз,Овощи. Фрукты. Грибы. Зелень,Бахчевые,1621,кг,11265.38,6.95,683772.7


In [90]:

shelves_df = raw_shelves_df
shelves_df['shelve'] = shelves_df['shelve'].apply(lambda x: ''.join(x.split('-')[:2]))

shelves_df.head()

,name,price,shelve
0,Апельсины,215,A7
1,Апельсины Навелин,289,A3
2,Арбуз,67,A7
3,Баклажаны грунтовые,138,Fr1
4,Бананы,158,A7


In [77]:

# The training data will look like this
# sku_id,	name,	avg_cart,	t_expiery, stocktaking_time,	dt,	trigger

columns = ['sku_id', 'name', 'avg_cart', 't_expiery', 'stocktaking_time', 'dt', 'trigger']

X, y = make_classification(n_samples=1000, n_features=6, n_informative=4, n_classes=2, random_state=42)

columns = ['avg_cart', 't_expiery', 'stocktaking_time', 'dt', 'feature_1', 'feature_2']
training_df = pd.DataFrame(X, columns=columns)

training_df['trigger'] = y

training_df.head()


   avg_cart  t_expiery  stocktaking_time        dt  feature_1  feature_2  \
0 -1.700306   0.546738          1.697493  1.020916  -3.568689   1.301264   
1  0.041412   1.157294          0.140774 -3.193412   2.094181  -1.021301   
2  1.161560  -0.397759         -1.908699 -1.800386   4.323467  -2.244412   
3 -1.486318   1.104109         -1.306469 -4.843622   3.572225  -2.327224   
4  1.916012  -1.199254         -2.594574 -0.640792   5.089422  -2.763972   

   trigger  
0        1  
1        0  
2        1  
3        0  
4        1  


In [79]:
X = training_df.drop(columns=['trigger'])
y = training_df['trigger']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = X_train.copy()
train_data['trigger'] = y_train


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [71]:
task = Task('binary')

automl = TabularAutoML(task=task, timeout=1200)

oof_pred = automl.fit_predict(train_data, roles={'target': 'trigger'})

test_pred = automl.predict(X_test)

print(test_pred.data[:10])

accuracy = accuracy_score(y_test, test_pred.data.round())
print(f"Точность модели: {accuracy:.2f}")

c:\code\vkusvill-ml\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


AssertionError: Pipeline finished with 0 models for some reason.
Probably one or more models failed